<a href="https://colab.research.google.com/github/Mpaaco/42-in-C/blob/main/Machine_SellerFreeze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gspread oauth2client ftfy regex tqdm
!pip install -q git+https://github.com/openai/CLIP.git

  Preparing metadata (setup.py) ... done


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import torch
import clip
from PIL import Image
import requests
from io import BytesIO

In [ ]:
# 3. Conecta ao Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

# Substitua pela sua URL da planilha
url_planilha = "https://docs.google.com/spreadsheets/d/1Z_ffAxu4c3ARKBwqUrNEk13p_Xhr6WqdCWzdJWGJSuA/edit?usp=sharing"
sheet = client.open_by_url(url_planilha).sheet1

# 4. Carrega o modelo CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [ ]:
def baixar_imagem(url):
    try:
        response = requests.get(url)
        return Image.open(BytesIO(response.content)).convert("RGB")
    except:
        return None

def comparar_imagens(url1, url2):
    img1 = baixar_imagem(url1)
    img2 = baixar_imagem(url2)
    if not img1 or not img2:
        return 0.0
    img1_tensor = preprocess(img1).unsqueeze(0).to(device)
    img2_tensor = preprocess(img2).unsqueeze(0).to(device)
    with torch.no_grad():
        emb1 = model.encode_image(img1_tensor)
        emb2 = model.encode_image(img2_tensor)
        emb1 /= emb1.norm(dim=-1, keepdim=True)
        emb2 /= emb2.norm(dim=-1, keepdim=True)
        similarity = (emb1 @ emb2.T).item()
    return similarity